In [97]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import seaborn as sns
import math
import os
from sklearn.model_selection import train_test_split

In [98]:
os.getcwd()

'/mnt/c/Users/12428/Documents/GitHub/MAT_180_ML_Projects/machine-learning-assisted-khovanov-homology'

In [3]:
os.chdir("..")

In [100]:
from scripts.polynomial import add_poly_terms
from scripts.GDLinearReg import J, DJ, GD_linreg_improved
from scripts.predictAccuracy import prediction, accuracy
from scripts.getGridsDimensions import find_max_min_jones

# Step1. Read Dataset

In [101]:
df = pd.read_csv('data/dataset_C.csv')

In [102]:
df

,braid,components,khovanov_homology,free_part,torsion_part,free_part_count,torsion_part_count,total_num_FP_per_row,total_num_FP_per_column,jones_polynomial
0,"[2, 2, 1, -2, 2, 1, -1, -1, 2]",1,"{1: {0: Z}, 3: {0: Z, 1: 0, 2: 0, 3: 0}, 5: {0...","{(1, 0): 1, (3, 0): 1, (5, 2): 1, (9, 3): 1}","{(7, 3): {2: 1}}",4,1,"{1: 1, 3: 1, 5: 1, 9: 1}","{0: 2, 2: 1, 3: 1}","{1: 1, 3: 1, 4: -1}"
1,"[-2, -1, 1, -3, -1, 2, -3, -2, -1]",1,"{-13: {-6: 0, -5: Z, -4: 0}, -11: {-6: 0, -5: ...","{(-13, -5): 1, (-9, -4): 1, (-9, -3): 1, (-7, ...","{(-11, -4): {2: 1}, (-7, -2): {2: 1}, (-5, -1)...",8,3,"{-13: 1, -9: 2, -7: 1, -5: 1, -3: 2, -1: 1}","{-5: 1, -4: 1, -3: 1, -2: 2, -1: 1, 0: 2}","{-6: -1, -5: 1, -4: -1, -3: 2, -2: -1, -1: 1}"
2,"[-3, -2, 3, -2, -1, -1, 1, -3, -1]",3,"{-15: {-6: Z}, -13: {-6: Z, -5: 0, -4: 0, -3: ...","{(-15, -6): 1, (-13, -6): 1, (-11, -4): 2, (-9...","{(-9, -3): {2: 1}, (-5, -1): {2: 1}}",12,2,"{-15: 1, -13: 1, -11: 2, -9: 1, -7: 3, -5: 1, ...","{-6: 2, -4: 3, -3: 1, -2: 3, -1: 1, 0: 2}","{-7: 1, -5: 2, -4: -1, -3: 2, -2: -1, -1: 1}"
3,"[3, 3, 2, 3, -1, -2, 2, 3, 3]",1,"{1: {-1: 0, 0: 0}, 3: {-1: 0, 0: Z, 1: 0, 2: 0...","{(3, 0): 1, (5, 0): 1, (7, 2): 1, (11, 3): 1, ...","{(9, 3): {2: 1}, (13, 5): {2: 1}}",6,2,"{3: 1, 5: 1, 7: 1, 11: 2, 15: 1}","{0: 2, 2: 1, 3: 1, 4: 1, 5: 1}","{2: 1, 4: 1, 5: -1, 6: 1, 7: -1}"
4,"[-3, -3, 2, -2, 2, 1, -3, -3, -3]",1,"{-15: {-5: Z}, -13: {-5: 0, -4: C2}, -11: {-5:...","{(-15, -5): 1, (-11, -4): 1, (-11, -3): 1, (-7...","{(-13, -4): {2: 1}, (-9, -2): {2: 1}}",6,2,"{-15: 1, -11: 2, -7: 1, -5: 1, -3: 1}","{-5: 1, -4: 1, -3: 1, -2: 1, 0: 2}","{-7: -1, -6: 1, -5: -1, -4: 1, -2: 1}"
...,...,...,...,...,...,...,...,...,...,...
2526,"[3, 1, 3, -1, -1, -1, -1, -1, -1]",3,"{-17: {-6: 0, -5: 0}, -15: {-6: 0, -5: Z, -4: ...","{(-15, -5): 1, (-13, -5): 1, (-11, -4): 1, (-1...","{(-13, -4): {2: 1}, (-11, -4): {2: 1}, (-9, -2...",24,8,"{-15: 1, -13: 1, -11: 3, -9: 3, -7: 3, -5: 4, ...","{-5: 2, -4: 2, -3: 4, -2: 4, -1: 2, 0: 6, 2: 4}","{-7: -1, -5: -1, -3: 1, -2: 1, -1: 2, 0: 1, 1: 1}"
2527,"[2, 2, 2, -1, -2, -3, -2, -1, -3]",1,"{-11: {-6: 0, -5: Z, -4: 0}, -9: {-6: 0, -5: 0...","{(-11, -5): 1, (-7, -4): 1, (-7, -3): 1, (-5, ...","{(-9, -4): {2: 1}, (-5, -2): {2: 1}, (-3, -1):...",10,4,"{-11: 1, -7: 2, -5: 1, -3: 1, -1: 3, 1: 1, 3: 1}","{-5: 1, -4: 1, -3: 1, -2: 2, -1: 1, 0: 3, 1: 1}","{-5: -1, -4: 1, -3: -1, -2: 2, -1: -1, 0: 2, 1..."
2528,"[2, 2, -1, 3, -1, -2, -2, -1, -1]",3,"{-15: {-6: Z}, -13: {-6: 0, -5: Z x C2, -4: 0}...","{(-15, -6): 1, (-13, -5): 1, (-11, -5): 1, (-1...","{(-13, -5): {2: 1}, (-11, -4): {2: 1}, (-7, -2...",20,6,"{-15: 1, -13: 1, -11: 3, -9: 5, -7: 2, -5: 2, ...","{-6: 1, -5: 2, -4: 5, -3: 2, -2: 4, -1: 2, 0: 4}","{-7: 1, -6: -2, -5: 3, -4: -2, -3: 4, -2: -2, ..."
2529,"[2, -3, -3, 2, -3, -1, -2, -1, 2]",1,"{-13: {-6: 0, -5: 0}, -11: {-6: 0, -5: 0, -4: ...","{(-9, -3): 1, (-5, -2): 1, (-3, 0): 1, (-1, 0)...","{(-7, -2): {2: 1}}",4,1,"{-9: 1, -5: 1, -3: 1, -1: 1}","{-3: 1, -2: 1, 0: 2}","{-4: -1, -3: 1, -1: 1}"


In [103]:
def fit(X, y, epsilon, lambda_, max_iters = 10000, poly_terms = 1):
    print(f'Running polynomial regression of degree {poly_terms} \n')
    
    v, costs =  GD_linreg_improved(add_poly_terms(X, poly_terms), y, epsilon, lambda_, max_iters) 
    
    print(f'\nFinal cost is {costs[-1]}\n')
    return v, costs

In [104]:
find_max, find_min = find_max_min_jones(df)
(find_max, find_min)

(13, -13)

In [105]:
# # only use this for 1-component links
# def gen_matrix(Jones,col):
#     row = len(Jones)
#     a = np.empty([row,col])
#     for i in range(row):
#         for key in Jones[i].keys():
#             a[i,key] =  Jones[i][key]
#         if key == "":
#             a[i,key] = 0
#     return a

# 
def gen_all_keys(jones,col):
    row = len(jones)
    a = []
    for i in range(row):
        for key in jones[i].keys():
            if key not in a:
                a.append(key)
    return a

# use this for multiple-components links
def gen_matrix_jones(jones,keys):
    row = len(jones)
    col = len(keys)
    m = np.zeros([row,col])
    for i in range(col):
        for j in range(len(jones)):
            for key,degree in jones[j].items():
                if key == keys[i]:
                    m[j,i] = degree
    return m

# vectorize the eval function
ev = np.vectorize(eval)

# Step2. Train model and make prediction using links with multiple-components

In [106]:
# prediciton with multiple-component link
Jones = ev(df["jones_polynomial"])

# get all the features
keys = gen_all_keys(Jones,find_max-find_min+1)
keys.sort()

In [107]:
X = gen_matrix_jones(Jones,keys)
y = np.array(df['torsion_part_count']).reshape(-1,1)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [109]:
degree = 1

v, costs = fit(X_train, y_train, epsilon = 1e-10, lambda_ = 0, max_iters = 1000, poly_terms = degree)

print(f'Accuracy of training set is: {accuracy(add_poly_terms(X_train, degree),v,y_train)}')
print(f'Accuracy of validation set is: {accuracy(add_poly_terms(X_val, degree),v,y_val)}')
print(f'Accuracy of test set is: {accuracy(add_poly_terms(X_test, degree),v,y_test)}')

Running polynomial regression of degree 1 

After 0 steps the cost is 37.69038208168644
After 999 steps the cost is 9.710083778338808

Final cost is 9.709620385051792

Accuracy of training set is: 0.1251646903820817
Accuracy of validation set is: 0.15612648221343872
Accuracy of test set is: 0.1341222879684418


In [45]:
# #try to get a dataframe and plot to see the relationships of features
# X_df = pd.DataFrame(X)
# X_df['torsion_part_count'] = df['torsion_part_count']
# X_df

In [46]:
# # took too long to generate the image
# sns.pairplot(X_df)

In [47]:
degree = 2

v, costs = fit(X_train, y_train, epsilon = 1e-10, lambda_ = 0, max_iters = 1000, poly_terms = degree)

print(f'Accuracy of training set is: {accuracy(add_poly_terms(X_train, degree),v,y_train)}')
print(f'Accuracy of validation set is: {accuracy(add_poly_terms(X_val, degree),v,y_val)}')
print(f'Accuracy of test set is: {accuracy(add_poly_terms(X_test, degree),v,y_test)}')

Running polynomial regression of degree 2 

After 0 steps the cost is 37.69038208168644
After 999 steps the cost is 3.111102935776602

Final cost is 3.1105636008279047

Accuracy of training set is: 0.3188405797101449
Accuracy of validation set is: 0.31225296442687744
Accuracy of test set is: 0.3431952662721893


# Step3. Train model and make prediction using links with 1-component only

In [114]:
# prediciton with 1-component link
Jones = ev(df[df.components == 1]["jones_polynomial"])

# get all the features
keys = gen_all_keys(Jones,find_max-find_min+1)
keys.sort()

In [115]:
X = gen_matrix_jones(Jones,keys)
y = np.array(df[df.components == 1]['torsion_part_count']).reshape(-1,1)

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [117]:
degree = 1

v, costs = fit(X_train, y_train, epsilon = 1e-10, lambda_ = 0, max_iters = 1000, poly_terms = degree)

print(f'Accuracy of training set is: {accuracy(add_poly_terms(X_train, degree),v,y_train)}')
print(f'Accuracy of validation set is: {accuracy(add_poly_terms(X_val, degree),v,y_val)}')
print(f'Accuracy of test set is: {accuracy(add_poly_terms(X_test, degree),v,y_test)}')

Running polynomial regression of degree 1 

After 0 steps the cost is 34.95317220543806
After 999 steps the cost is 7.918113981433843

Final cost is 7.918055053643977

Accuracy of training set is: 0.12084592145015106
Accuracy of validation set is: 0.13122171945701358
Accuracy of test set is: 0.08597285067873303


# Step4. Train model and make prediction using links with 2-components only

In [118]:
# prediciton with 2-components link
Jones = ev(df[df.components == 2]["jones_polynomial"])

# get all the features
keys = gen_all_keys(Jones,find_max-find_min+1)
keys.sort()

In [119]:
X = gen_matrix_jones(Jones,keys)
y = np.array(df[df.components == 2]['torsion_part_count']).reshape(-1,1)

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [121]:
degree = 1

v, costs = fit(X_train, y_train, epsilon = 1e-10, lambda_ = 0, max_iters = 1000, poly_terms = degree)

print(f'Accuracy of training set is: {accuracy(add_poly_terms(X_train, degree),v,y_train)}')
print(f'Accuracy of validation set is: {accuracy(add_poly_terms(X_val, degree),v,y_val)}')
print(f'Accuracy of test set is: {accuracy(add_poly_terms(X_test, degree),v,y_test)}')

Running polynomial regression of degree 1 

After 0 steps the cost is 27.086956521739136
After 999 steps the cost is 7.317935403175287

Final cost is 7.317850215663211

Accuracy of training set is: 0.20652173913043478
Accuracy of validation set is: 0.16129032258064516
Accuracy of test set is: 0.1774193548387097


# Step5. Train model and make prediction using links with 3-components only

In [122]:
# prediciton with 3-components link
Jones = ev(df[df.components == 3]["jones_polynomial"])

# get all the features
keys = gen_all_keys(Jones,find_max-find_min+1)
keys.sort()

In [123]:
X = gen_matrix_jones(Jones,keys)
y = np.array(df[df.components == 3]['torsion_part_count']).reshape(-1,1)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [125]:
degree = 1

v, costs = fit(X_train, y_train, epsilon = 1e-10, lambda_ = 0, max_iters = 1000, poly_terms = degree)

print(f'Accuracy of training set is: {accuracy(add_poly_terms(X_train, degree),v,y_train)}')
print(f'Accuracy of validation set is: {accuracy(add_poly_terms(X_val, degree),v,y_val)}')
print(f'Accuracy of test set is: {accuracy(add_poly_terms(X_test, degree),v,y_test)}')

Running polynomial regression of degree 1 

After 0 steps the cost is 38.46795827123695
After 999 steps the cost is 9.970865349347234

Final cost is 9.970569137043912

Accuracy of training set is: 0.18777943368107303
Accuracy of validation set is: 0.21875
Accuracy of test set is: 0.19196428571428573


# Step6. Train the Regression model from Scikit-Learn and make prediction using links with multiple-components

In [75]:
# try to make prediction using LinearRegression and libraries from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report

# import libraries for Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures

In [76]:
# function to compute accuracy
def find_accuracy(y_pred, y_true):
    accur_count = 0
    for index,y in enumerate(y_pred):
        if y == math.ceil(y_true[index]):
            accur_count += 1
    return accur_count/len(y_true)

In [77]:
# Find the degree of Polynomial Regression that gives the best prediction accuracy
model_degrees_and_scores = {}
model_degrees_and_accuracy = {}
for current_degree in range(1,4):
    polynomial_features= PolynomialFeatures(degree=current_degree)
    X_poly_train_deg = polynomial_features.fit_transform(X_train)
    print(f'Polynomial Features: [1,x,x**2,...,x**{current_degree}]')
    print(X_poly_train_deg[0])
    
    X_poly_val_deg = polynomial_features.fit_transform(X_val)
    print(f'Polynomial Features: [1,x,x**2,...,x**{current_degree}]')
    print(X_poly_val_deg[0])
    

    X_poly_test_deg = polynomial_features.fit_transform(X_test)
    print(f'Polynomial Features: [1,x,x**2,...,x**{current_degree}]')
    print(X_poly_test_deg[0])

    polyreg = LinearRegression().fit(X_poly_train_deg, y_train)
    
    train_score = polyreg.score(X_poly_train_deg,y_train)
    val_score = polyreg.score(X_poly_val_deg,y_val)
    test_score = polyreg.score(X_poly_test_deg,y_test)
        
    y_train_pred = polyreg.predict(X_poly_train_deg)
    y_val_pred = polyreg.predict(X_poly_val_deg)
    y_test_pred = polyreg.predict(X_poly_test_deg)
    train_accuracy = find_accuracy(y_train, y_train_pred)
    val_accuracy = find_accuracy(y_val, y_val_pred)
    test_accuracy = find_accuracy(y_test, y_test_pred)
    
    model_degrees_and_scores[current_degree] = (train_score, val_score, test_score)
    model_degrees_and_accuracy[current_degree] = (train_accuracy, val_accuracy, test_accuracy)

Polynomial Features: [1,x,x**2,...,x**1]
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  2.  0. -2.  0.
  2.  0. -2.  0.  2.  0. -1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Polynomial Features: [1,x,x**2,...,x**1]
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0. -1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Polynomial Features: [1,x,x**2,...,x**1]
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  0.  1.
  0. -2.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Polynomial Features: [1,x,x**2,...,x**2]
[1. 0. 0. ... 0. 0. 0.]
Polynomial Features: [1,x,x**2,...,x**2]
[1. 0. 0. ... 0. 0. 0.]
Polynomial Features: [1,x,x**2,...,x**2]
[1. 0. 0. ... 0. 0. 0.]
Polynomial Features: [1,x,x**2,...,x**3]
[1. 0

In [78]:
for i, accuracy in model_degrees_and_accuracy.items():
    print(f'Degree {i} scores: {accuracy}')

Degree 1 scores: (0.1080368906455863, 0.13043478260869565, 0.126232741617357)
Degree 2 scores: (0.20619235836627142, 0.22727272727272727, 0.22682445759368836)
Degree 3 scores: (0.2635046113306983, 0.23517786561264822, 0.20118343195266272)
